## 1. 資料清理
因為我有更新過 namelist 資料表，所以要先撈出來植群資料庫(nvdimp_plots_w_sp)沒對到的物種，為了方便起見，
另外也再新增一個欄位 namecode，儲存 TaiBNET 的 namecode，未來若物種有分類異動的話只要更新 namecode 即可。

In [1]:
import psycopg2 as pg
import psycopg2.extras
import csv
import re
import itertools
import os
from IPython import display
DSN = 'dbname=nvdimp user=psilotum'
conn = pg.connect(DSN)

In [6]:
with conn:
        with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as curs:
            CHECK_NULL_ZHNAME = '''
            SELECT 
                distinct species,s.zh_name
            FROM 
                nvdimp_plots_w_sp s 
            LEFT OUTER JOIN 
                namelist n 
            ON 
                s.species= n.name 
            WHERE n.zh_name is null; '''
            curs.execute(CHECK_NULL_ZHNAME)
            null_names = curs.fetchall()

結果顯示有 151 個學名沒有正確對應到中名，因此使用 PostgreSQL fuzzymatch 的 ```metaphone()``` 來搜尋比對，因為有 151 個學名要比對，所以寫成 function 來加速比對的程序

In [11]:
class ListTable(list):
    # 參考 
    # http://calebmadrigal.com/display-list-as-table-in-ipython-notebook/
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
def fuzzysearch(spname, metaphone_sensitivity=7, **kwargs):
    print('檢查的學名為: %s' % spname)
    # get data from namelist
    META_PHONE_MATCH =  '''
        SELECT 
            namecode,name,zh_name[1] 
        FROM 
            namelist 
        WHERE 
            metaphone('%s', %i) = metaphone(name,%i);''' % (spname, metaphone_sensitivity, metaphone_sensitivity)
    with conn:
        with conn.cursor(cursor_factory = psycopg2.extras.DictCursor) as curs:
            curs.execute(META_PHONE_MATCH)
            sp_candidates = curs.fetchall()
    sp_candidate_container = []
    global table
    table = ListTable()
    table.append(['id', 'namecode','name','zh_name'])
    for i in range(0, len(sp_candidates)):
        sp_row = list(itertools.chain(str(i), sp_candidates[i]))
        sp_candidate_container.append(sp_row)
        table.append(sp_row)
    print('相似的學名如下： \n')
    display.display(table)
    
    # choose correct species namecode
    nid = input('Select nearest candidate id: ')

    if nid == '':
        spname_correct = input('Please input the correct name: ')
        UPDATE_MANUALLY = '''
        UPDATE nvdimp_plots_w_sp
        SET 
            species = '%s' 
        WHERE
            species = '%s'; ''' % (spname_correct, spname)
        with conn:
            with conn.cursor(cursor_factory = psycopg2.extras.DictCursor) as curs:
                curs.execute(UPDATE_MANUALLY)
    else:
        # update table
        UPDATE_TAB = '''
        UPDATE nvdimp_plots_w_sp
        SET 
            namecode = '%s', species = '%s'
        WHERE species = '%s'; 
        ''' % (table[int(nid) + 1][1], table[int(nid) + 1][2], spname)
        print(UPDATE_TAB)
        confirm = input('Confirm? (Y/n) ')
        if confirm == 'Y' or confirm == 'y':
            with conn:
                with conn.cursor(cursor_factory = psycopg2.extras.DictCursor) as curs:
                    curs.execute(UPDATE_TAB)
        else:
            print('Please type Y or y!')
            pass
    Out.clear()

In [215]:
for sp in range(103, len(null_names)):
    print(sp)
    fuzzysearch(null_names[sp]['species'])
    Out.clear()

103
檢查的學名為: Lycopodium serratum
相似的學名如下： 



id,namecode,name,zh_name
0,205632,Lycopodium cernuum,過山龍


Select nearest candidate id: 
Please input the correct name: Huperzia serrata
104
檢查的學名為: Adiantum capillusveneris
相似的學名如下： 



id,namecode,name,zh_name
0,205023,Adiantum capillus-junonis,團羽鐵線蕨
1,205024,Adiantum capillus-veneris,鐵線蕨


Select nearest candidate id: 1

        UPDATE nvdimp_plots_w_sp
        SET 
            namecode = '205024', species = 'Adiantum capillus-veneris'
        WHERE species = 'Adiantum capillusveneris'; 
        
Confirm? (Y/n) y
105
檢查的學名為: Citrus depress
相似的學名如下： 



...

Select nearest candidate id: 0

        UPDATE nvdimp_plots_w_sp
        SET 
            namecode = '204164', species = 'Potentilla matsumurae'
        WHERE species = 'Potentilla matsumurae var. pilosa'; 
        
Confirm? (Y/n) y
149
檢查的學名為: Oxalis acetocella subsp. taemoni
相似的學名如下： 



id,namecode,name,zh_name
0,203412,Oxalis acetosella subsp. griffithii,山酢漿草
1,203412,Oxalis acetosella subsp. formosana,臺灣山酢漿草
2,430803,Oxalis acetosella subsp. taemoni,大霸尖山酢漿草
3,430804,Oxalis acetosella subsp. griffithii,臺灣山酢漿草


Select nearest candidate id: 2

        UPDATE nvdimp_plots_w_sp
        SET 
            namecode = '430803', species = 'Oxalis acetosella subsp. taemoni'
        WHERE species = 'Oxalis acetocella subsp. taemoni'; 
        
Confirm? (Y/n) y
150
檢查的學名為: Cunninghamia konishii
相似的學名如下： 



id,namecode,name,zh_name


Select nearest candidate id: 
Please input the correct name: Cunninghamia lanceolata var. konishii


In [255]:
# 再 check 一次
with conn:
    with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as curs:
        CHECK_NULL_ZHNAME = '''
        SELECT 
            distinct species,s.zh_name
        FROM 
            nvdimp_plots_w_sp s 
        LEFT OUTER JOIN 
            namelist n 
        ON 
            s.species= n.name 
        WHERE s.zh_name is null; '''
        curs.execute(CHECK_NULL_ZHNAME)
        null_names = curs.fetchall()

In [256]:
null_names

[]

## 2. 空間上物種分布點位與保護區的交集

### 資料前處理

因為植群樣區調查的資料有調查年份，但之前取得的資料並沒有完整(例：Braun-Blanquet 樣區調查資料中有 pdate 的日期，
但傳統樣區我取得的資料表並無，所以得找看看之前的 rawdata 有沒有資料）

In [266]:
# 發現沒有年份，重新撈資料回來更新
nvdimp_m_plotdata = []
with open('./data/nvdimp_m_plotdata_200812.csv', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        nvdimp_m_plotdata.append(row)

In [278]:
for i in range(1, len(nvdimp_m_plotdata)):
    UPDATE_NVDIMP_M_DATE = '''
    UPDATE 
        nvdimp_tradplots_info
    SET 
        survey_year = %i
    WHERE 
        plotid = '%s';''' % (int(nvdimp_m_plotdata[i][38]), nvdimp_m_plotdata[i][1])
    with conn:
        with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as curs:
            curs.execute(UPDATE_NVDIMP_M_DATE)

In [9]:
# 更新調查年份(survey_year)

# Braun-Blanquet 樣區調查中使用 pdate 當成調查日期，新建一個 survey_year 欄位，
# 並根據 pdate 填入資料
# 增加調查年份的 column SQL: ALTER TABLE nvdimp_plots_w_sp add COLUMN survey_year integer;
#
#
# 
# nvdimp_plots_w_sp: 每個樣區的物種清單
# nvdimp_brbqplots_info: Braun-Blanquet 樣區 header 資訊
# nvdimp_tradplots_info: 傳統英美學派計數樣區法
UPDATE_SURVEY_YR = '''UPDATE 
    nvdimp_brbqplots_info 
SET 
    survey_year = cast(split_part(cast(pdate as text), '-', 1) as INTEGER);
UPDATE 
    nvdimp_plots_w_sp as s 
SET 
    survey_year = t.survey_year 
FROM 
    (SELECT * FROM nvdimp_tradplots_info) as t 
WHERE t.plotid = s.plotid;
UPDATE 
    nvdimp_plots_w_sp as s 
SET 
    survey_year = t.survey_year 
FROM 
    (SELECT * FROM nvdimp_brbqplots_info) as t 
WHERE t.plotid = s.plotid;'''
with conn:
    with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as curs:
        curs.execute(UPDATE_SURVEY_YR)

更新之後，稍微驗算一下資料更新是否有問題

In [12]:
# 確認看是否有調查年份是空值(null)者
CHECK_YR_NULL = "SELECT  plotid FROM nvdimp_plots_w_sp WHERE survey_year is null;"
CHECK_YEARLY_PLOTS_NUM = '''SELECT 
    f.survey_year,count(plotid) 
FROM 
 (SELECT distinct plotid, survey_year FROM nvdimp_plots_w_sp) AS f 
GROUP BY 
    f.survey_year order by survey_year;'''
with conn:
    with conn.cursor() as curs:
        curs.execute(CHECK_YR_NULL)
        check_null_res = curs.fetchall()
        if  check_null_res == '':
            print("OK!")
        else:
            check_null_res
        curs.execute(CHECK_YEARLY_PLOTS_NUM)
        yr_plots_no = curs.fetchall()
        table = ListTable()
        for i in yr_plots_no:
            table.append(i)
            


In [13]:
table

2003,699
2004,737
2005,708
2006,700
2007,720


看起來每年大概做了 700 個樣區(真的很辛苦，印象中 2007 年和定宏做樣區時候中央山脈走七天還做不到 20 個)

## 套用國家公園的圖層來計算物種清單

接下來我們使用 PostGIS ```st_intersects(geom_a, geom_b)``` 來找出國家植群計畫樣區有哪些是在國家公園及自然保護留區系統中，就能夠建立各國家公園及自然保護留區系統的植物名錄


相關圖層：

| 圖層名稱      | 縮寫 | 我的 postgis 資料表名稱   | 資料筆數   | 備註     |
| ------------ | ----|---------------------- | --------: |---------|
| 國家公園      | NPK   | env.tw_national_park    | 9        |         |
| 國家自然公園   | NNP  | NA                      | 1        | 尚待建立 |
| 自然保護區     | NPA  |env.tw_nature_protect   | 6        | |
| 自然保留區     | NRA  |env.tw_nature_reserve   | 19       | |
| 野生動物重要棲息環境| WLH | env.tw_wildlife_habitats | 33  | |
| 野生動物保護區  | WLP  | env.tw_wildlife_protect | 17       | |

In [53]:
def intersect_geolyrs(lyr, splist = 'nvdimp_plots_w_sp', lyr_geom = 'geom_wgs84', \
                      splist_geom = 'geom_wgs84', **kwargs):
    
    output_list = []
    with conn:
        with conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor) as curs:
            QUERY_SPYR = '''SELECT 
                  distinct np.name, p.zh_name, p.species, p.survey_year
               FROM 
                  env.%s np, %s p 
               WHERE 
                  st_intersects(np.%s, p.%s)
               ORDER BY np.name, p.survey_year;''' % (lyr, splist, lyr_geom, splist_geom)
            curs.execute(QUERY_SPYR)
            res = curs.fetchall()
            for i in res:
                output_list.append([i['name'], i['survey_year'], i['species'], i['zh_name']])
    return(output_list)

In [50]:
npsys = ['tw_national_park', 'tw_nature_protect', 'tw_nature_reserve', 'tw_wildlife_habitats',\
        'tw_wildlife_protect']
for n in npsys:
    outlist = intersect_geolyrs(n)
    outfile = open("nvdimp_%s.csv" % n,'w')
    wr = csv.writer(outfile, dialect='excel')
    wr.writerows(outlist)

TypeError: list indices must be integers, not str

In [32]:
# 靠么， pandas 壞掉了，懶得修，以下開始改用 R
import rpy2.ipython
import rpy2.robjects.packages as rpackages


/usr/local/lib/python3.4/site-packages/rpy2/ipython/rmagic.py:68: UserWarning: The Python package 'pandas' is stronglyrecommended when using `rpy2.ipython`. Unfortunately it could not be loaded, but at least we found 'numpy'.
  "but at least we found 'numpy'.")))


In [34]:
%load_ext rpy2.ipython

In [46]:
%%R -o a
a = read.csv('./data/checklists/tw_national_park.csv', header=F)
colnames(a) = c('np_name', 'year', 'species', 'zh_name')
head(a)


       np_name year                               species zh_name
1 墾丁國家公園 2004              Lagerstroemia subcostata    九芎
2 墾丁國家公園 2004                  Cryptocarya concinna    土楠
3 墾丁國家公園 2004              Parthenocissus dalzielii    地錦
4 墾丁國家公園 2004 Capparis sikkimensis subsp. formosana    山柑
5 墾丁國家公園 2004                  Champereia manillana    山柚
6 墾丁國家公園 2004                        Arenga tremula    山棕


In [52]:
outlist

['玉里野生動物保護區', 2006, 'Miscanthus sinensis', '芒']